In [1]:
import pandas as pd
import numpy as np

# Start coding here...

bank = pd.read_csv("bank_marketing.csv")
print(bank.head())

   client_id  age        job  ... euribor3m nr_employed   y
0          0   56  housemaid  ...     4.857      5191.0  no
1          1   57   services  ...     4.857      5191.0  no
2          2   37   services  ...     4.857      5191.0  no
3          3   40     admin.  ...     4.857      5191.0  no
4          4   56   services  ...     4.857      5191.0  no

[5 rows x 22 columns]


In [2]:
# Splitting the dataframe into three tables

client = bank[
    [
        "client_id",
        "duration",
        "age",
        "job",
        "marital",
        "education",
        "credit_default",
        "housing",
        "loan",
    ]
]

campaign = bank[
    [
        "client_id",
        "campaign",
        "month",
        "day",
        "duration",
        "pdays",
        "previous",
        "poutcome",
        "y",
    ]
]

economics = bank[
    ["client_id", "emp_var_rate", "cons_price_idx", "euribor3m", "nr_employed"]
]


In [3]:
# renaming columns

client = client.rename(columns={"client_id": "id"})

campaign = campaign.rename(
    columns={
        "duration": "content_duration",
        "previous": "previous_campaign_contacts",
        "y": "campaign_outcome",
        "campaign": "number_contacts",
        "poutcome": "previous_outcome",
    }
)

economics = economics.rename(
    columns={"euribor3m": "euribor_three_months", "nr_employed": "number_employed"}
)

In [4]:
# cleaning data

client["education"] = (
    client["education"].str.replace(".", "_").replace("unknown", np.NaN)
)

client["job"] = client["job"].str.replace(".", "")

campaign["previous_outcome"] = campaign["previous_outcome"].map(
    {"failure": 0, "success": 1, "nonexistent": np.NaN}
)

campaign["campaign_outcome"] = campaign["campaign_outcome"].map(
    {"failure": 0, "success": 1}
)

campaign["campaign_id"] = 1

campaign["last_contact_date"] = (
    "2022"
    + "-"
    + campaign["month"].str.capitalize()
    + "-"
    + campaign["day"].astype(str)
)

campaign["last_contact_date"] = pd.to_datetime(
    campaign["last_contact_date"], format="%Y-%b-%d"
)

campaign.drop(["month", "day"], axis=1, inplace=True)

In [5]:
#creating a csv
client.to_csv('client.csv', index=False)

campaign.to_csv('campaign.csv', index=False)

economics.to_csv('economics.csv', index=False)

In [7]:
clientdf = pd.read_csv('client.csv')
print(clientdf.head())

   id  duration  age        job  ...    education credit_default housing loan
0   0       261   56  housemaid  ...     basic_4y             no      no   no
1   1       149   57   services  ...  high_school        unknown      no   no
2   2       226   37   services  ...  high_school             no     yes   no
3   3       151   40      admin  ...     basic_6y             no      no   no
4   4       307   56   services  ...  high_school             no      no  yes

[5 rows x 9 columns]


In [8]:
campaigndf = pd.read_csv('campaign.csv')
print(campaigndf.head())

   client_id  number_contacts  ...  campaign_id  last_contact_date
0          0                1  ...            1         2022-05-13
1          1                1  ...            1         2022-05-19
2          2                1  ...            1         2022-05-23
3          3                1  ...            1         2022-05-27
4          4                1  ...            1         2022-05-03

[5 rows x 9 columns]


In [9]:
economicsdf = pd.read_csv('economics.csv')
print(economicsdf.head())

   client_id  emp_var_rate  ...  euribor_three_months  number_employed
0          0           1.1  ...                 4.857           5191.0
1          1           1.1  ...                 4.857           5191.0
2          2           1.1  ...                 4.857           5191.0
3          3           1.1  ...                 4.857           5191.0
4          4           1.1  ...                 4.857           5191.0

[5 rows x 5 columns]


In [6]:
#creating tables for the datebase
client_table = """CREATE TABLE client
(
    id SERIAL PRIMARY KEY,
    content_duration INTERGER,
    age INTEGER,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN
);
\copy client from 'client.csv' DELIMITER ',' CSV HEADER
"""

campaign_table = """CREATE TABLE campaign
(
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL references client (id),
    number_contacts INTEGER,
    contact_duration INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATE    
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""

economics_table = """CREATE TABLE economics
(
    client_id SERIAL references client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""